In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

ada= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
cc= SMOTE(random_state= 15)

X_ada, y_ada= ada.fit_resample(X_train, y_train)
X_ada_cc, y_ada_cc= cc.fit_resample(X_ada, y_ada)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_ada_cc= LogisticRegression(random_state=15)
lr_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_lr_ada_cc= lr_ada_cc.predict(X_test)
y_pred_prob_lr_ada_cc= lr_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_ada_cc, labels= lr_ada_cc.classes_)
print(classification_report(y_test, y_pred_lr_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_lr_ada_cc))

              precision    recall  f1-score   support

           0       0.97      0.72      0.82      8517
           1       0.23      0.80      0.36       912

    accuracy                           0.72      9429
   macro avg       0.60      0.76      0.59      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.23171889838556506
Accuracy: 0.7235125676105632
AUC: 0.7588360430841105


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_ada_cc= DecisionTreeClassifier(random_state=15)
dt_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_dt_ada_cc= dt_ada_cc.predict(X_test)
y_pred_prob_dt_ada_cc= dt_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_ada_cc, labels= dt_ada_cc.classes_)
print(classification_report(y_test, y_pred_dt_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_ada_cc))

              precision    recall  f1-score   support

           0       0.95      0.13      0.23      8517
           1       0.10      0.93      0.19       912

    accuracy                           0.21      9429
   macro avg       0.52      0.53      0.21      9429
weighted avg       0.86      0.21      0.22      9429

Precision: 0.10280939694841366
Accuracy: 0.20755117191642805
AUC: 0.5305069041483597


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_ada_cc= GridSearchCV(estimator= dt_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_dt_ada_cc.best_params_)
print("Best Precision Score:", gs_dt_ada_cc.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 20}
Best Precision Score: 0.9298955400135334


In [12]:
tuned_dt_ada_cc= gs_dt_ada_cc.best_estimator_
y_pred_tuned_dt_ada_cc= tuned_dt_ada_cc.predict(X_test)
y_pred_prob_tuned_dt_ada_cc= tuned_dt_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_ada_cc, labels= tuned_dt_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_dt_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_ada_cc))

              precision    recall  f1-score   support

           0       0.95      0.12      0.21      8517
           1       0.10      0.95      0.19       912

    accuracy                           0.20      9429
   macro avg       0.53      0.53      0.20      9429
weighted avg       0.87      0.20      0.21      9429

Precision: 0.10283941780004772
Accuracy: 0.19715770495280519
AUC: 0.5276409899499246


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_ada_cc= KNeighborsClassifier()
knn_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_knn_ada_cc= knn_ada_cc.predict(X_test)
y_pred_prob_knn_ada_cc= knn_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_ada_cc, labels= knn_ada_cc.classes_)
print(classification_report(y_test, y_pred_knn_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_ada_cc))

              precision    recall  f1-score   support

           0       0.93      0.84      0.88      8517
           1       0.23      0.44      0.30       912

    accuracy                           0.80      9429
   macro avg       0.58      0.64      0.59      9429
weighted avg       0.86      0.80      0.83      9429

Precision: 0.22513966480446929
Accuracy: 0.7989182309895004
AUC: 0.7288750672030551


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_ada_cc= GridSearchCV(estimator= knn_ada_cc, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_knn_ada_cc.best_params_)
print("Best Precision Score:", gs_knn_ada_cc.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 6, 'weights': 'uniform'}
Best Precision Score: 0.8409497600561116


In [15]:
tuned_knn_ada_cc= gs_knn_ada_cc.best_estimator_
y_pred_tuned_knn_ada_cc= tuned_knn_ada_cc.predict(X_test)
y_pred_prob_tuned_knn_ada_cc= tuned_knn_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_ada_cc, labels= tuned_knn_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_knn_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_ada_cc))

              precision    recall  f1-score   support

           0       0.97      0.62      0.75      8517
           1       0.19      0.82      0.31       912

    accuracy                           0.64      9429
   macro avg       0.58      0.72      0.53      9429
weighted avg       0.89      0.64      0.71      9429

Precision: 0.18734429496761335
Accuracy: 0.6370771025559444
AUC: 0.779189750014934


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_ada_cc= GaussianNB()
nb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_nb_ada_cc= nb_ada_cc.predict(X_test)
y_pred_prob_nb_ada_cc= nb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_ada_cc, labels= nb_ada_cc.classes_)
print(classification_report(y_test, y_pred_nb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_ada_cc))

              precision    recall  f1-score   support

           0       0.98      0.39      0.56      8517
           1       0.14      0.91      0.24       912

    accuracy                           0.44      9429
   macro avg       0.56      0.65      0.40      9429
weighted avg       0.90      0.44      0.53      9429

Precision: 0.1387639513576545
Accuracy: 0.4433131827341181
AUC: 0.7942315832730822


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_ada_cc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_svc_ada_cc= svc_ada_cc.predict(X_test)
y_pred_prob_svc_ada_cc= svc_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_ada_cc, labels= svc_ada_cc.classes_)
print(classification_report(y_test, y_pred_svc_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_ada_cc))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      8517
           1       0.13      0.07      0.09       912

    accuracy                           0.86      9429
   macro avg       0.52      0.51      0.51      9429
weighted avg       0.83      0.86      0.85      9429

Precision: 0.1324110671936759
Accuracy: 0.8638243716194718
AUC: 0.5905758143156412


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_ada_cc= RandomForestClassifier(random_state=15)
rf_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_rf_ada_cc= rf_ada_cc.predict(X_test)
y_pred_prob_rf_ada_cc= rf_ada_cc.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_ada_cc, labels= rf_ada_cc.classes_)
print(classification_report(y_test, y_pred_rf_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_ada_cc))

              precision    recall  f1-score   support

           0       0.99      0.05      0.10      8517
           1       0.10      1.00      0.18       912

    accuracy                           0.15      9429
   macro avg       0.55      0.53      0.14      9429
weighted avg       0.91      0.15      0.11      9429

Precision: 0.10133928571428572
Accuracy: 0.14561459327606321
AUC: 0.6481129588089044


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_ada_cc= GridSearchCV(estimator= rf_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_tuned_rf_ada_cc.best_params_)
print("Best Precision Score:", gs_tuned_rf_ada_cc.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Best Precision Score: 0.9430357014826211


In [21]:
tuned_rf_ada_cc= gs_tuned_rf_ada_cc.best_estimator_
y_pred_tuned_rf_ada_cc= tuned_rf_ada_cc.predict(X_test)
y_pred_prob_tuned_rf_ada_cc= tuned_rf_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_ada_cc, labels= tuned_rf_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_rf_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_ada_cc))

              precision    recall  f1-score   support

           0       0.99      0.06      0.12      8517
           1       0.10      1.00      0.18       912

    accuracy                           0.15      9429
   macro avg       0.55      0.53      0.15      9429
weighted avg       0.91      0.15      0.12      9429

Precision: 0.10196518809657495
Accuracy: 0.1514476614699332
AUC: 0.6625985001102026


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_ada_cc= AdaBoostClassifier(random_state=15)
ada_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_ada_ada_cc= ada_ada_cc.predict(X_test)
y_pred_prob_ada_ada_cc= ada_ada_cc.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_ada_cc, labels= ada_ada_cc.classes_)
print(classification_report(y_test, y_pred_ada_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_ada_cc))

              precision    recall  f1-score   support

           0       0.96      0.04      0.07      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.12      9429
weighted avg       0.87      0.13      0.08      9429

Precision: 0.09851892484914976
Accuracy: 0.12705483084102237
AUC: 0.6223332488789192


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_ada_cc= GridSearchCV(estimator= ada_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_ada_cc.fit(X_ada_cc, y_ada_cc)
print("Best Parameters:", gs_ada_ada_cc.best_params_)
print("Best Precision Score:", gs_ada_ada_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9588618020863139


In [25]:
tuned_ada_ada_cc= gs_ada_ada_cc.best_estimator_
y_pred_tuned_ada_ada_cc= tuned_ada_ada_cc.predict(X_test)
y_pred_prob_tuned_ada_ada_cc= tuned_ada_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_ada_cc, labels= tuned_ada_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_ada_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_ada_cc))

              precision    recall  f1-score   support

           0       0.95      0.03      0.06      8517
           1       0.10      0.99      0.18       912

    accuracy                           0.12      9429
   macro avg       0.52      0.51      0.12      9429
weighted avg       0.87      0.12      0.07      9429

Precision: 0.09805846422338568
Accuracy: 0.12164598578852476
AUC: 0.6175940817024361


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_ada_cc= GradientBoostingClassifier(random_state=15)
grb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_grb_ada_cc= grb_ada_cc.predict(X_test)
y_pred_prob_grb_ada_cc= grb_ada_cc.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_ada_cc, labels= grb_ada_cc.classes_)
print(classification_report(y_test, y_pred_grb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_ada_cc))

              precision    recall  f1-score   support

           0       0.97      0.04      0.07      8517
           1       0.10      0.99      0.18       912

    accuracy                           0.13      9429
   macro avg       0.54      0.51      0.12      9429
weighted avg       0.89      0.13      0.08      9429

Precision: 0.09906747120131651
Accuracy: 0.12811538869445327
AUC: 0.6396919139018145


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_ada_cc= GridSearchCV(estimator= grb_ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_grb_ada_cc.best_params_)
print("Best Precision Score:", gs_grb_ada_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9565317923922055


In [28]:
tuned_grb_ada_cc= gs_grb_ada_cc.best_estimator_
y_pred_tuned_grb_ada_cc= tuned_grb_ada_cc.predict(X_test)
y_pred_prob_tuned_grb_ada_cc= tuned_grb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_ada_cc, labels= tuned_grb_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_grb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_ada_cc))

              precision    recall  f1-score   support

           0       0.95      0.04      0.07      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.12      9429
weighted avg       0.87      0.13      0.08      9429

Precision: 0.0984092155787164
Accuracy: 0.1268427192703362
AUC: 0.6004169421734447


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_ada_cc= XGBClassifier(random_state=15)
xgb_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_xgb_ada_cc= xgb_ada_cc.predict(X_test)
y_pred_prob_xgb_ada_cc= xgb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_ada_cc, labels= xgb_ada_cc.classes_)
print(classification_report(y_test, y_pred_xgb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_ada_cc))

              precision    recall  f1-score   support

           0       0.95      0.03      0.06      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.12      9429
   macro avg       0.52      0.51      0.12      9429
weighted avg       0.87      0.12      0.07      9429

Precision: 0.0982045106196628
Accuracy: 0.12482765934881748
AUC: 0.5915397018141221


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_ada_cc= GridSearchCV(estimator= xgb_ada_cc, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_xgb_ada_cc.best_params_)
print("Best Precision Score:", gs_xgb_ada_cc.best_score_)

tuned_xgb_ada_cc= gs_xgb_ada_cc.best_estimator_
y_pred_tuned_xgb_ada_cc= tuned_xgb_ada_cc.predict(X_test)
y_pred_prob_tuned_xgb_ada_cc= tuned_xgb_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_ada_cc, labels= tuned_xgb_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_ada_cc))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 1.0}
Best Precision Score: 0.956671582636144
              precision    recall  f1-score   support

           0       0.95      0.04      0.08      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.87      0.13      0.09      9429

Precision: 0.09870015421899096
Accuracy: 0.13055467175734436
AUC: 0.5753062373704603


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_ada_cc= LGBMClassifier(random_state=15)
lgm_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_lgm_ada_cc= lgm_ada_cc.predict(X_test)
y_pred_prob_lgm_ada_cc= lgm_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_ada_cc, labels= lgm_ada_cc.classes_)
print(classification_report(y_test, y_pred_lgm_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_ada_cc))

[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.474579 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2594
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.96      0.04      0.07      8517
           1       0.10      0.99      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.88      0.13      0.08      9429

Precision: 0.0988998899889989
Accuracy: 0.12991833704528583
AUC: 0.5742551275158662


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_ada_cc= GridSearchCV(estimator= lgm_ada_cc, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_lgm_ada_cc.best_params_)
print("Best Precision Score:", gs_lgm_ada_cc.best_score_)

tuned_lgm_ada_cc= gs_lgm_ada_cc.best_estimator_
y_pred_tuned_lgm_ada_cc= tuned_lgm_ada_cc.predict(X_test)
y_pred_prob_tuned_lgm_ada_cc= tuned_lgm_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_ada_cc, labels= tuned_lgm_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_ada_cc))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003401 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2594
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

              precision    recall  f1-score   support

           0       0.96      0.04      0.08      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.88      0.13      0.09      9429

Precision: 0.09911602209944752
Accuracy: 0.1337363453176371
AUC: 0.5963379613322375


--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_ada_cc= CatBoostClassifier(random_state=15)
cat_ada_cc.fit(X_ada_cc, y_ada_cc)
y_pred_cat_ada_cc= cat_ada_cc.predict(X_test)
y_pred_prob_cat_ada_cc= cat_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_ada_cc, labels= cat_ada_cc.classes_)
print(classification_report(y_test, y_pred_cat_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_ada_cc))

Learning rate set to 0.024073
0:	learn: 0.6551040	total: 145ms	remaining: 2m 25s
1:	learn: 0.6211057	total: 154ms	remaining: 1m 17s
2:	learn: 0.5862167	total: 161ms	remaining: 53.6s
3:	learn: 0.5584884	total: 170ms	remaining: 42.4s
4:	learn: 0.5273643	total: 178ms	remaining: 35.4s
5:	learn: 0.4977605	total: 188ms	remaining: 31.1s
6:	learn: 0.4731720	total: 195ms	remaining: 27.7s
7:	learn: 0.4530026	total: 203ms	remaining: 25.2s
8:	learn: 0.4327317	total: 210ms	remaining: 23.1s
9:	learn: 0.4132794	total: 218ms	remaining: 21.6s
10:	learn: 0.3955118	total: 226ms	remaining: 20.3s
11:	learn: 0.3778764	total: 235ms	remaining: 19.3s
12:	learn: 0.3642001	total: 246ms	remaining: 18.6s
13:	learn: 0.3504747	total: 264ms	remaining: 18.6s
14:	learn: 0.3391590	total: 274ms	remaining: 18s
15:	learn: 0.3265705	total: 281ms	remaining: 17.3s
16:	learn: 0.3152379	total: 291ms	remaining: 16.8s
17:	learn: 0.3041745	total: 299ms	remaining: 16.3s
18:	learn: 0.2941196	total: 307ms	remaining: 15.9s
19:	learn: 

163:	learn: 0.0853814	total: 4.89s	remaining: 25s
164:	learn: 0.0851830	total: 4.93s	remaining: 25s
165:	learn: 0.0849835	total: 4.98s	remaining: 25s
166:	learn: 0.0847968	total: 4.99s	remaining: 24.9s
167:	learn: 0.0846229	total: 5.02s	remaining: 24.9s
168:	learn: 0.0842975	total: 5.04s	remaining: 24.8s
169:	learn: 0.0841223	total: 5.08s	remaining: 24.8s
170:	learn: 0.0839237	total: 5.1s	remaining: 24.7s
171:	learn: 0.0837614	total: 5.14s	remaining: 24.7s
172:	learn: 0.0833147	total: 5.16s	remaining: 24.7s
173:	learn: 0.0830904	total: 5.19s	remaining: 24.6s
174:	learn: 0.0828703	total: 5.23s	remaining: 24.7s
175:	learn: 0.0826030	total: 5.25s	remaining: 24.6s
176:	learn: 0.0823193	total: 5.29s	remaining: 24.6s
177:	learn: 0.0821659	total: 5.33s	remaining: 24.6s
178:	learn: 0.0820077	total: 5.34s	remaining: 24.5s
179:	learn: 0.0818529	total: 5.36s	remaining: 24.4s
180:	learn: 0.0816705	total: 5.41s	remaining: 24.5s
181:	learn: 0.0815027	total: 5.45s	remaining: 24.5s
182:	learn: 0.08130

325:	learn: 0.0599970	total: 9.82s	remaining: 20.3s
326:	learn: 0.0599056	total: 9.85s	remaining: 20.3s
327:	learn: 0.0598051	total: 9.87s	remaining: 20.2s
328:	learn: 0.0596998	total: 9.88s	remaining: 20.1s
329:	learn: 0.0595705	total: 9.89s	remaining: 20.1s
330:	learn: 0.0594964	total: 9.91s	remaining: 20s
331:	learn: 0.0593756	total: 9.93s	remaining: 20s
332:	learn: 0.0592254	total: 9.97s	remaining: 20s
333:	learn: 0.0591453	total: 9.98s	remaining: 19.9s
334:	learn: 0.0590493	total: 9.99s	remaining: 19.8s
335:	learn: 0.0589369	total: 10s	remaining: 19.8s
336:	learn: 0.0588175	total: 10s	remaining: 19.7s
337:	learn: 0.0587530	total: 10s	remaining: 19.6s
338:	learn: 0.0586561	total: 10s	remaining: 19.6s
339:	learn: 0.0585703	total: 10s	remaining: 19.5s
340:	learn: 0.0584704	total: 10.1s	remaining: 19.4s
341:	learn: 0.0582533	total: 10.1s	remaining: 19.4s
342:	learn: 0.0581375	total: 10.1s	remaining: 19.3s
343:	learn: 0.0579640	total: 10.1s	remaining: 19.2s
344:	learn: 0.0577623	total:

498:	learn: 0.0428853	total: 11.7s	remaining: 11.7s
499:	learn: 0.0428393	total: 11.7s	remaining: 11.7s
500:	learn: 0.0427834	total: 11.7s	remaining: 11.6s
501:	learn: 0.0427298	total: 11.7s	remaining: 11.6s
502:	learn: 0.0426695	total: 11.7s	remaining: 11.6s
503:	learn: 0.0425738	total: 11.7s	remaining: 11.5s
504:	learn: 0.0425098	total: 11.7s	remaining: 11.5s
505:	learn: 0.0424464	total: 11.7s	remaining: 11.5s
506:	learn: 0.0423619	total: 11.8s	remaining: 11.4s
507:	learn: 0.0422830	total: 11.8s	remaining: 11.4s
508:	learn: 0.0422169	total: 11.8s	remaining: 11.4s
509:	learn: 0.0421098	total: 11.8s	remaining: 11.3s
510:	learn: 0.0420553	total: 11.8s	remaining: 11.3s
511:	learn: 0.0419635	total: 11.8s	remaining: 11.3s
512:	learn: 0.0419006	total: 11.8s	remaining: 11.2s
513:	learn: 0.0417654	total: 11.8s	remaining: 11.2s
514:	learn: 0.0416863	total: 11.8s	remaining: 11.1s
515:	learn: 0.0416433	total: 11.8s	remaining: 11.1s
516:	learn: 0.0415833	total: 11.9s	remaining: 11.1s
517:	learn: 

659:	learn: 0.0330827	total: 15.5s	remaining: 7.98s
660:	learn: 0.0330261	total: 15.5s	remaining: 7.96s
661:	learn: 0.0329859	total: 15.6s	remaining: 7.94s
662:	learn: 0.0329334	total: 15.6s	remaining: 7.92s
663:	learn: 0.0328893	total: 15.6s	remaining: 7.9s
664:	learn: 0.0328233	total: 15.6s	remaining: 7.87s
665:	learn: 0.0327908	total: 15.7s	remaining: 7.85s
666:	learn: 0.0327061	total: 15.7s	remaining: 7.83s
667:	learn: 0.0326848	total: 15.7s	remaining: 7.82s
668:	learn: 0.0325181	total: 15.7s	remaining: 7.79s
669:	learn: 0.0324841	total: 15.8s	remaining: 7.77s
670:	learn: 0.0324407	total: 15.8s	remaining: 7.75s
671:	learn: 0.0323762	total: 15.8s	remaining: 7.73s
672:	learn: 0.0323024	total: 15.9s	remaining: 7.7s
673:	learn: 0.0322389	total: 15.9s	remaining: 7.68s
674:	learn: 0.0321984	total: 15.9s	remaining: 7.66s
675:	learn: 0.0321472	total: 16s	remaining: 7.65s
676:	learn: 0.0321067	total: 16s	remaining: 7.62s
677:	learn: 0.0320810	total: 16s	remaining: 7.59s
678:	learn: 0.032036

820:	learn: 0.0260944	total: 19.4s	remaining: 4.24s
821:	learn: 0.0260676	total: 19.5s	remaining: 4.21s
822:	learn: 0.0260264	total: 19.5s	remaining: 4.19s
823:	learn: 0.0259988	total: 19.5s	remaining: 4.17s
824:	learn: 0.0259595	total: 19.5s	remaining: 4.14s
825:	learn: 0.0259255	total: 19.5s	remaining: 4.12s
826:	learn: 0.0258668	total: 19.6s	remaining: 4.09s
827:	learn: 0.0258351	total: 19.6s	remaining: 4.07s
828:	learn: 0.0258049	total: 19.6s	remaining: 4.04s
829:	learn: 0.0257785	total: 19.6s	remaining: 4.02s
830:	learn: 0.0257315	total: 19.6s	remaining: 3.99s
831:	learn: 0.0257050	total: 19.7s	remaining: 3.97s
832:	learn: 0.0256713	total: 19.7s	remaining: 3.94s
833:	learn: 0.0256509	total: 19.7s	remaining: 3.92s
834:	learn: 0.0256217	total: 19.7s	remaining: 3.89s
835:	learn: 0.0255988	total: 19.7s	remaining: 3.87s
836:	learn: 0.0255725	total: 19.7s	remaining: 3.85s
837:	learn: 0.0255360	total: 19.8s	remaining: 3.82s
838:	learn: 0.0254916	total: 19.8s	remaining: 3.8s
839:	learn: 0

984:	learn: 0.0211385	total: 21.8s	remaining: 332ms
985:	learn: 0.0211070	total: 21.8s	remaining: 310ms
986:	learn: 0.0210889	total: 21.8s	remaining: 287ms
987:	learn: 0.0210735	total: 21.8s	remaining: 265ms
988:	learn: 0.0210514	total: 21.8s	remaining: 243ms
989:	learn: 0.0210114	total: 21.8s	remaining: 221ms
990:	learn: 0.0209835	total: 21.9s	remaining: 199ms
991:	learn: 0.0209351	total: 21.9s	remaining: 176ms
992:	learn: 0.0209137	total: 21.9s	remaining: 154ms
993:	learn: 0.0208929	total: 21.9s	remaining: 132ms
994:	learn: 0.0208736	total: 21.9s	remaining: 110ms
995:	learn: 0.0208557	total: 21.9s	remaining: 87.9ms
996:	learn: 0.0208384	total: 21.9s	remaining: 65.9ms
997:	learn: 0.0208111	total: 21.9s	remaining: 43.9ms
998:	learn: 0.0207893	total: 21.9s	remaining: 21.9ms
999:	learn: 0.0207678	total: 21.9s	remaining: 0us
              precision    recall  f1-score   support

           0       0.96      0.03      0.06      8517
           1       0.10      0.99      0.18       912

  

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_ada_cc= GridSearchCV(estimator= cat_ada_cc, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_ada_cc.fit(X_ada_cc, y_ada_cc)

print("Best Parameters:", gs_cat_ada_cc.best_params_)
print("Best Precision Score:", gs_cat_ada_cc.best_score_)

tuned_cat_ada_cc= gs_cat_ada_cc.best_estimator_
y_pred_tuned_cat_ada_cc= tuned_cat_ada_cc.predict(X_test)
y_pred_prob_tuned_cat_ada_cc= tuned_cat_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_ada_cc, labels= tuned_cat_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_cat_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_ada_cc))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.3163154	total: 9.88ms	remaining: 978ms
1:	learn: 0.2579225	total: 17.3ms	remaining: 850ms
2:	learn: 0.2095640	total: 25.3ms	remaining: 818ms
3:	learn: 0.1589419	total: 33.7ms	remaining: 810ms
4:	learn: 0.1424265	total: 50.3ms	remaining: 956ms
5:	learn: 0.1310730	total: 60ms	remaining: 940ms
6:	learn: 0.1196299	total: 67ms	remaining: 891ms
7:	learn: 0.1107604	total: 75.2ms	remaining: 864ms
8:	learn: 0.1064296	total: 82.8ms	remaining: 837ms
9:	learn: 0.0923161	total: 89.9ms	remaining: 809ms
10:	learn: 0.0886917	total: 98.2ms	remaining: 794ms
11:	learn: 0.0852090	total: 106ms	remaining: 780ms
12:	learn: 0.0835114	total: 114ms	remaining: 762ms
13:	learn: 0.0805719	total: 132ms	remaining: 813ms
14:	learn: 0.0785556	total: 139ms	remaining: 790ms
15:	learn: 0.0771017	total: 146ms	remaining: 768ms
16:	learn: 0.0752892	total: 153ms	remaining: 749ms
17:	learn: 0.0733521	total: 165ms	remaining: 753ms
18:	learn: 0.0681626	to

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_ada_cc, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_ada_cc, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_ada_cc, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_ada_cc, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_ada_cc, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_ada_cc, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_ada_cc, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_ada_cc, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_ada_cc, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_ada_cc, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_ada_cc, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division= 0))

Logistic Regression Precision: 0.23171889838556506
Decision Tree Precision: 0.10280939694841366
Tuned Decision Tree Precision: 0.10283941780004772
KNeighborsClassifier Precision: 0.22513966480446929
Tuned KNeighborsClassifier Precision: 0.18734429496761335
GaussianNB Precision: 0.1387639513576545
SVM Precision: 0.1324110671936759
Random Forest Precision: 0.10133928571428572
Tuned Random Forest Precision: 0.10196518809657495
AdaBoost Precision: 0.09851892484914976
Tuned AdaBoost Precision: 0.09805846422338568
GradientBoosting Precision: 0.09906747120131651
Tuned GradientBoosting Precision: 0.0984092155787164
XGB Precision: 0.0982045106196628
Tuned XGB Precision: 0.09870015421899096
LGBM Precision: 0.0988998899889989
Tuned LGBM Precision: 0.09911602209944752
CatBoost Precision: 0.0986640385457731
Tuned CatBoost Precision: 0.0979551451187335


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_ada_cc, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_ada_cc, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_ada_cc, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_ada_cc, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_ada_cc, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_ada_cc, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_ada_cc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_ada_cc, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_ada_cc, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_ada_cc, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_ada_cc, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_ada_cc, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_ada_cc, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_ada_cc, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_ada_cc, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_ada_cc, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_ada_cc, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_ada_cc, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_ada_cc, zero_division= 0)
                  }

ada_cc_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
ada_cc_precision= ada_cc_precision.sort_values(by= 'Precision Score', ascending=False)
print(ada_cc_precision)

                                    Model  Precision Score
0          Logistic Regression Precision:         0.231719
3         KNeighborsClassifier Precision:         0.225140
4   Tuned KNeighborsClassifier Precision:         0.187344
5                   GaussianNB Precision:         0.138764
6                          SVM Precision:         0.132411
2          Tuned Decision Tree Precision:         0.102839
1                Decision Tree Precision:         0.102809
8          Tuned Random Forest Precision:         0.101965
7                Random Forest Precision:         0.101339
16                  Tuned LGBM Precision:         0.099116
11            GradientBoosting Precision:         0.099067
15                        LGBM Precision:         0.098900
14                   Tuned XGB Precision:         0.098700
17                    CatBoost Precision:         0.098664
9                     AdaBoost Precision:         0.098519
12      Tuned GradientBoosting Precision:         0.0984

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_ada_cc),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_ada_cc),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_ada_cc),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_ada_cc),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_ada_cc),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_ada_cc),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_ada_cc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_ada_cc),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_ada_cc),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_ada_cc),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_ada_cc),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_ada_cc),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_ada_cc),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_ada_cc),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_ada_cc),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_ada_cc),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_ada_cc),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_ada_cc),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_ada_cc)
                  }

ada_cc_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
ada_cc_accuracy= ada_cc_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(ada_cc_accuracy)

                                   Model  Accuracy Score
6                          SVM Accuracy:        0.863824
3         KNeighborsClassifier Accuracy:        0.798918
0          Logistic Regression Accuracy:        0.723513
4   Tuned KNeighborsClassifier Accuracy:        0.637077
5                   GaussianNB Accuracy:        0.443313
1                Decision Tree Accuracy:        0.207551
2          Tuned Decision Tree Accuracy:        0.197158
8          Tuned Random Forest Accuracy:        0.151448
7                Random Forest Accuracy:        0.145615
16                  Tuned LGBM Accuracy:        0.133736
14                   Tuned XGB Accuracy:        0.130555
15                        LGBM Accuracy:        0.129918
11            GradientBoosting Accuracy:        0.128115
18              Tuned CatBoost Accuracy:        0.127585
9                     AdaBoost Accuracy:        0.127055
12      Tuned GradientBoosting Accuracy:        0.126843
17                    CatBoost 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_ada_cc),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_ada_cc),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_ada_cc),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_ada_cc),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_ada_cc),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_ada_cc),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_ada_cc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_ada_cc),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_ada_cc),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_ada_cc),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_ada_cc),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_ada_cc),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_ada_cc),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_ada_cc),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_ada_cc),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_ada_cc),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_ada_cc),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_ada_cc),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_ada_cc)
                  }

ada_cc_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
ada_cc_auc= ada_cc_auc.sort_values(by= 'AUC Score', ascending=False)
print(ada_cc_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.831153
5                   GaussianNB AUC:   0.794232
4   Tuned KNeighborsClassifier AUC:   0.779190
3         KNeighborsClassifier AUC:   0.728875
8          Tuned Random Forest AUC:   0.662599
7                Random Forest AUC:   0.648113
11            GradientBoosting AUC:   0.639692
17                    CatBoost AUC:   0.624706
9                     AdaBoost AUC:   0.622333
10              Tuned AdaBoost AUC:   0.617594
12      Tuned GradientBoosting AUC:   0.600417
16                  Tuned LGBM AUC:   0.596338
13                         XGB AUC:   0.591540
6                          SVM AUC:   0.590576
18              Tuned CatBoost AUC:   0.584381
14                   Tuned XGB AUC:   0.575306
15                        LGBM AUC:   0.574255
1                Decision Tree AUC:   0.530507
2          Tuned Decision Tree AUC:   0.527641
